<a href="https://colab.research.google.com/github/vinay-jose/rag-nbs/blob/main/PaliGemma_Pdf_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install poppler-utils
!sudo apt-get install tesseract-ocr

In [ ]:
!pip install  -Uqq accelerate bitsandbytes git+https://github.com/huggingface/transformers.git # model dependencies

In [ ]:
!pip install -Uqq PyMuPDF pytesseract pdf2image # pdf ingestion dependencies

In [4]:
import numpy as np
from PIL import Image
import requests

In [5]:
from transformers import AutoTokenizer, PaliGemmaForConditionalGeneration, PaliGemmaProcessor
from transformers import BitsAndBytesConfig
import torch

In [6]:
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [7]:
device="cuda"
model_id = "google/paligemma-3b-mix-224"
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.bfloat16,
                                                          quantization_config=nf4_config, device_map={"":0})
processor = PaliGemmaProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

In [8]:
import fitz  # PyMuPDF
import pytesseract
from pdf2image import convert_from_path

In [10]:
def extract_text_from_pdf(pdf_path):
    # Try extracting text directly
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()

    # If no text was extracted, use OCR
    if not text.strip():
        images = convert_from_path(pdf_path)
        for image in images:
            text += pytesseract.image_to_string(image)

    return text

# Usage
pdf_text = extract_text_from_pdf("/content/THINKZ schedule of work.pdf")
# print(pdf_text)

In [11]:
images = convert_from_path("/content/THINKZ schedule of work.pdf")
# for image in images:
#   display(image)

In [12]:
input_text = "What is the total amount?"

In [13]:
inputs = processor(text=input_text, images=images[-1],
                  padding="longest", do_convert_rgb=True, return_tensors="pt").to("cuda")
# model.to(device)
inputs = inputs.to(dtype=model.dtype)

In [14]:
with torch.no_grad():
  output = model.generate(**inputs, max_length=496)

print(processor.decode(output[0], skip_special_tokens=True))

What is the total amount?
$11,631.50
